In [13]:
#!pip install catboost

In [2]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold
#warnings.filterwarnings('ignore')
#%matplotlib inline
from sklearn.metrics import roc_auc_score

In [3]:
## 数据导入
from sklearn.model_selection import train_test_split  
from catboost import CatBoostClassifier
train=pd.read_csv("data/data63592/train.csv")
testA=pd.read_csv("data/data63592/testA.csv")
print('数据导入成功！')

数据导入成功！


In [5]:
numerical_fea = list(train.select_dtypes(exclude=['object']).columns)
numerical_fea.remove('isDefault')
numerical_fea.remove('n2')

# from fancyimpute import KNN
# train = KNN(k=5).fit_transform(train[numerical_fea])
# testA = KNN(k=5).fit_transform(testA[numerical_fea])
# train = pd.DataFrame(train)
# testA = pd.DataFrame(testA)
# print(train.head())
# print(testA.head())

train[numerical_fea] = train[numerical_fea].fillna(train[numerical_fea].median())
testA[numerical_fea] = testA[numerical_fea].fillna(testA[numerical_fea].median())

# train[numerical_fea] = train[numerical_fea].dropna()

In [6]:
#数据预处理
for data in [train]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})
    data['employmentLength'] = data['employmentLength'].map({'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'10+ years':10,'< 1 year':0})
    data['subGrade'] = data['subGrade'].map({'E2':1,'D2':2,'D3':3,'A4':4,'C2':5,'A5':6,'C3':7,'B4':8,'B5':9,'E5':10,
        'D4':11,'B3':12,'B2':13,'D1':14,'E1':15,'C5':16,'C1':17,'A2':18,'A3':19,'B1':20,
        'E3':21,'F1':22,'C4':23,'A1':24,'D5':25,'F2':26,'E4':27,'F3':28,'G2':29,'F5':30,
        'G3':31,'G1':32,'F4':33,'G4':34,'G5':35})
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))
  #  data['n15']=data['n8']*data['n10']
    
for data in [testA]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})
    data['employmentLength'] = data['employmentLength'].map({'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'10+ years':10,'< 1 year':0})
    data['subGrade'] = data['subGrade'].map({'E2':1,'D2':2,'D3':3,'A4':4,'C2':5,'A5':6,'C3':7,'B4':8,'B5':9,'E5':10,
        'D4':11,'B3':12,'B2':13,'D1':14,'E1':15,'C5':16,'C1':17,'A2':18,'A3':19,'B1':20,
        'E3':21,'F1':22,'C4':23,'A1':24,'D5':25,'F2':26,'E4':27,'F3':28,'G2':29,'F5':30,
        'G3':31,'G1':32,'F4':33,'G4':34,'G5':35})
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

print("数据预处理完成!")  

数据预处理完成!


In [7]:
sub=testA[['id']].copy()
sub['isDefault']=0
testA=testA.drop(['id','issueDate'],axis=1)
data_x=train.drop(['isDefault','id','issueDate'],axis=1)
data_y=train[['isDefault']].copy()
x, val_x, y, val_y = train_test_split(  
    data_x,  
    data_y,  
    test_size=0.2,  
    random_state=1,  
    stratify=data_y
)  

In [8]:
col=['grade','subGrade','employmentTitle','homeOwnership','verificationStatus','purpose','postCode','regionCode',
     'initialListStatus','applicationType','policyCode']
for i in data_x.columns:
    if i in col:
        data_x[i] = data_x[i].astype('str')
for i in testA.columns:
    if i in col:
        testA[i] = testA[i].astype('str')

In [11]:
model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            task_type="GPU",
            learning_rate=0.1,
            iterations=500,
            random_seed=2020,
            od_type="Iter",
            depth=7)


In [12]:
import pandas as pd
answers = []
mean_score = 0
n_folds = 5
sk = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2021)
for train, test in sk.split(data_x, data_y):
    x_train = data_x.iloc[train]
    y_train = data_y.iloc[train]
    x_test = data_x.iloc[test]
    y_test = data_y.iloc[test]
    clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=100,cat_features=col)
    yy_pred_valid=clf.predict(x_test)
    print('cat验证的auc:{}'.format(roc_auc_score(y_test, yy_pred_valid)))
    mean_score += roc_auc_score(y_test, yy_pred_valid) / n_folds
    y_pred_valid = clf.predict(testA,prediction_type='Probability')[:,-1]
    answers.append(y_pred_valid)
print('mean valAuc:{}'.format(mean_score))
cat_pre=sum(answers)/n_folds
sub['isDefault']=cat_pre
sub.to_csv('data/submit.csv',index=False) 

0:	learn: 0.5608061	test: 0.5727825	best: 0.5727825 (0)	total: 58.3ms	remaining: 29.1s
100:	learn: 0.7183899	test: 0.7184298	best: 0.7184298 (100)	total: 5.16s	remaining: 20.4s
200:	learn: 0.7251087	test: 0.7227840	best: 0.7227840 (200)	total: 10.4s	remaining: 15.5s
300:	learn: 0.7285709	test: 0.7244499	best: 0.7244499 (300)	total: 15.7s	remaining: 10.4s
400:	learn: 0.7311564	test: 0.7251973	best: 0.7252351 (399)	total: 21s	remaining: 5.19s
499:	learn: 0.7331063	test: 0.7255938	best: 0.7256294 (488)	total: 26.4s	remaining: 0us
bestTest = 0.7256293595
bestIteration = 488
Shrink model to first 489 iterations.
cat验证的auc:0.5356385371560265
0:	learn: 0.5612352	test: 0.5564913	best: 0.5564913 (0)	total: 56.9ms	remaining: 28.4s
100:	learn: 0.7188204	test: 0.7172583	best: 0.7172583 (100)	total: 5.33s	remaining: 21.1s
200:	learn: 0.7259205	test: 0.7219132	best: 0.7219132 (200)	total: 10.7s	remaining: 15.9s
300:	learn: 0.7292400	test: 0.7236129	best: 0.7236129 (300)	total: 16.1s	remaining: 10.6s